In [1]:
from __future__ import print_function
import torch
import numpy as np

In [2]:
# 可以使用torch.nn 包来构建神经网络.
# 你已知道autograd 包, nn 包依赖autograd 包来定义模型并求导.一个nn.Module 包含各个层和
# 一个forward(input) 方法,该方法返回output

In [6]:
# 定义网络
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5*5 square convolutions

        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an offine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):

        # Max Pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)


Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [7]:
# 你只需定义forward 函数, backward 函数(计算梯度)在使用autograd 时自动为你创建.你可以
# 在forward 函数中使用Tensor 的任何操作。
# net.parameters() 返回模型需要学习的参数。
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [12]:
# forward 的输入和输出都是autograd.Variable .注意:这个网络(LeNet)期望的输入大小是
# 32*32.如果使用MNIST数据集来训练这个网络,请把图片大小重新调整到32*32.
# 通过网络中层输入维度的大小可以限制整个网络输入图像的大小
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.1459,  0.0626, -0.0970, -0.1241,  0.1518,  0.0531,  0.0645, -0.0942,
          0.0983,  0.0204]], grad_fn=<AddmmBackward>)


In [13]:
# 将所有参数的梯度缓存清零，然后进行随机梯度的反向传播
net.zero_grad()
out.backward(torch.randn(1, 10))

In [14]:
# 注意:
# torch.nn 只支持小批量输入,整个torch.nn 包都只支持小批量样本,而不支持单个样本
# 例如, nn.Conv2d 将接受一个4维的张量,每一维分别是
# (样本数*通道数*高*宽).
# 如果你有单个样本,只需使用input.unsqueeze(0) 来添加其它的维数

# 回顾
# torch.Tensor -支持自动编程操作（如backward() ）的多维数组。同时保持梯度的张量。
# nn.Module -神经网络模块.封装参数,移动到GPU上运行,导出,加载等
# nn.Parameter -一种张量,当把它赋值给一个Module 时,被自动的注册为参数.
# autograd.Function -实现一个自动求导操作的前向和反向定义, 每个张量操作都会创建至
# 少一个Function 节点，该节点连接到创建张量并对其历史进行编码的函数。


In [19]:
# 损失函数
# 现在,你反向跟踪loss ,使用它的.grad_fn 属性,你会看到向下面这样的一个计算图:
# input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d -> view -> linear -> relu ->
# linear -> relu -> linear -> MSELoss -> loss
# 所以, 当你调用loss.backward() ,整个图被区分为损失以及图中所有具有requires_grad = Tru
# e 的张量，并且其.grad 张量的梯度累积。
output = net(input)
target = torch.randn(10) # a dummy target, for example
target = target.view(1, -1) # make it the same shape as output
criterion = nn.MSELoss() #使用MSE损失函数
loss = criterion(output, target)
print(loss.grad_fn) #MSELoss
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [20]:
# 反向传播
# 为了反向传播误差,我们所需做的是调用loss.backward() .你需要清除已存在的梯度,否则梯度将
# 被累加到已存在的梯度。
# 现在,我们将调用loss.backward() ,并查看conv1层的偏置项在反向传播前后的梯度。
net.zero_grad() # 清楚已经存在的梯度
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)
loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)


conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0051, -0.0075, -0.0031,  0.0082, -0.0150,  0.0013])


In [22]:
# 更新权重
# 实践中最简单的更新规则是随机梯度下降(SGD)．weight=weight−learning_rate∗gradient
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

# 然而,当你使用神经网络是,你想要使用各种不同的更新规则,比如SGD,Nesterov-SGD , Adam , RMS
# PROP 等.为了能做到这一点,我们构建了一个包torch.optim 实现了所有的这些规则.使用他们非常简单:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01) # create your optimizer

# in your training loop:
optimizer.zero_grad() # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # Doed the update